In [ ]:
import argparse
import mxnet as mx
import os
import pandas as pd
import pickle
import re
import requests

from matplotlib import pyplot as plt
from mxnet.gluon.data.vision import datasets

import rsna_heme as rsna

In [ ]:
# 13377624, 10020048864, 10020076309, 10020076351
acc = '10020076351'

endpoint = 'http://localhost:8082/predictions/heme'
wl=[(40, 80), (80, 200), (40, 380)]

args = argparse.Namespace()
args.data_dir = '/mnt/Data2/datasets/rsna_heme/triple100/'

In [ ]:
mri = rsna.app.HemeStudy(acc = acc)
mri.setup()
mri.download(URL='https://air.radiology.ucsf.edu/api/', cred_path='/home/jcolby/.air_login.txt')
mri.setup()

In [ ]:
print(mri)

In [ ]:
dir_series = mri.series_to_path(2)
dcm_names = os.listdir(dir_series)

In [ ]:
dcm_names = sorted(dcm_names, key = lambda x: int(re.sub(".*\.(.*)\.dcm", "\\1", x)))

In [ ]:
dcm_path = os.path.join(dir_series, dcm_names[21])

In [ ]:
dcm = rsna.dicom.Dicom(dcm_path)

In [ ]:
fig = plt.figure(figsize=(512,512), dpi=1)
ax = fig.add_axes([0,0,1,1])
ax.axis('off')
dcm.plot()

In [ ]:
def _load_dcm(dcm_path):
    dcm = rsna.dicom.Dicom(dcm_path)
    img = dcm.img_for_plot3(wl)
    img, _ = rsna.transforms.common_transform(mx.nd.array(img), 0)
    img = img.flip(axis=2)
    data = mx.gluon.data.SimpleDataset([(img, 0)])
    data = data.transform_first(rsna.transforms.train_transform)
    return data

In [ ]:
probs_all = []
for dcm_name in dcm_names:
    dcm_path = os.path.join(dir_series, dcm_name)
    data = _load_dcm(dcm_path)
    data_str = pickle.dumps(data[0][0])
    prob = requests.post(endpoint, files = {'data': data_str}).json()
    probs_all.append(prob)
probs = pd.DataFrame(probs_all, columns=rsna.labels.heme_types)

In [ ]:
probs

In [ ]:
dcm_name = dcm_names[21]
dcm_path = os.path.join(dir_series, dcm_name)
data = _load_dcm(dcm_path)
plt.hist(data[0][0][2].asnumpy().flatten())

In [ ]:
fig = plt.figure(figsize=(512,512), dpi=1)
ax = fig.add_axes([0,0,1,1])
ax.axis('off')
ax.imshow(data[0][0][2].asnumpy(), cmap='gray')

In [ ]:
mri.rm_tmp()

***
# Confirm inference is working by testing a subject from the RSNA dataset

In [ ]:
i = 68556

In [ ]:
test_dataset = datasets.ImageRecordDataset(os.path.join(args.data_dir, 'stage_2_test.rec'), flag=1, transform=rsna.transforms.common_transform)

In [ ]:
fig = plt.figure(figsize=(512,512), dpi=1)
ax = fig.add_axes([0,0,1,1])
ax.axis('off')
ax.imshow(test_dataset[i][0][:,:,2].asnumpy(), cmap='gray')

In [ ]:
test_dataset = test_dataset.transform_first(rsna.transforms.train_transform)

In [ ]:
data_str = pickle.dumps(test_dataset[i][0])
requests.post(endpoint, files = {'data': data_str}).json()

In [ ]:
plt.hist(test_dataset[i][0][2].asnumpy().flatten())